In [3]:
import pandas as pd
from flashtext import KeywordProcessor
import spacy
from spacy.matcher import PhraseMatcher
import sys
import re

#progress bar packages
from tqdm import tqdm
#ngram package
from nltk.util import ngrams
from nltk.tokenize import RegexpTokenizer
import nltk

pd.set_option('display.max_columns', None)

#for downloading spacy stuff
#!{sys.executable} -m spacy download en

#en is the large model
nlp = spacy.load("en_core_web_sm")

In [4]:
df_sentences = pd.read_csv("/Users/ivowings/Sync/Thesis/Datasources/Preprocessed/sentences/sentences_emscad.csv")
df_softskills = pd.read_csv("/Users/ivowings/Sync/Thesis/Datasources/Preprocessed/Skills/softskills.csv")
df_hardskills = pd.read_csv("/Users/ivowings/Sync/Thesis/Datasources/Preprocessed/Skills/emsi_skills.csv")

df_hardskills = df_hardskills[df_hardskills['type']== 'Hard Skill']
df_hardskills = pd.DataFrame(df_hardskills['name'])

#soft_skill,soft_skill_lemmatized,soft_skill_no_stopwords
columnname = 'soft_skill_lemmatized'
df_softskills = df_softskills[[columnname]]

#Renameing the skill column names and merging afterwards
df_softskills = df_softskills.rename(columns={columnname: 'skill'})
df_hardskills = df_hardskills.rename(columns={'name': 'skill'})
df_skills = pd.concat([df_softskills, df_hardskills])
df_skills = df_skills.drop_duplicates()
df_skills

#Creating all the possible grams for each sentence
allgrams = []

#This tokenizer immediately removes punctuation and special characters from the sentence
tokenizer = RegexpTokenizer(r'\w+')

#,sentence,sentence_lemmatized,sentence_no_stopwords

for sentence in tqdm(df_sentences['sentence_lemmatized']):
    tokenizedsentence = tokenizer.tokenize(str(sentence))
    
    #Getting all possible n grams of the sentence
    #for n in range(1,len(tokenizedsentence)+1):
    
    #getting up to four grams for each sentence
    for n in range(1,5):
        grams = ngrams(tokenizedsentence,n)
        for gram in grams:
            allgrams.append(str(gram))

allgrams = pd.DataFrame(allgrams)
allgrams = allgrams.rename(columns={0:'allgrams'})
print(allgrams.shape[0])

#Initializing the keyword processor
keyword_processor = KeywordProcessor(case_sensitive=False)

#Adding all the skills to the processor
for skill in tqdm(df_skills['skill']):
    keyword_processor.add_keyword(skill)

def searcher(row):
    #check if the words are in the row and return a True or False instead of the actual word
    boolean = bool(keyword_processor.extract_keywords(row))
    return boolean

tqdm.pandas()
allgrams['contains_skill'] = allgrams['allgrams'].progress_apply(searcher)

#Only selecting the ngrams which contain a skill
allgrams = allgrams[allgrams.contains_skill == True]
#cleaning up
allgrams['allgrams'] = allgrams['allgrams'].astype(str)
allgrams['allgrams'] = allgrams['allgrams'].str.replace(r"[(),.']", '')
allgrams['allgrams'] = allgrams['allgrams'].str.strip()
allgrams = allgrams.drop(columns=['contains_skill'])

#Some softskills match multiple times, therefore removing the duplicates
allgrams = allgrams.drop_duplicates().reset_index(drop=True)

#removing any nans
allgrams.dropna(subset=['allgrams'],inplace=True)

import csv
allgrams.to_csv("/Users/ivowings/Sync/Thesis/Datasources/Preprocessed/Combined/Taxonomy/Lemmatized/n-grams.csv", quoting=csv.QUOTE_NONNUMERIC, index=False)

  0%|          | 0/28709 [00:00<?, ?it/s]

16433802


100%|██████████| 16433802/16433802 [02:57<00:00, 92801.19it/s] 
<ipython-input-4-2f4ebe647bc9>:62: FutureWarning: The default value of regex will change from True to False in a future version.
  allgrams['allgrams'] = allgrams['allgrams'].str.replace(r"[(),.']", '')


In [3]:
df_sentences = pd.read_csv("/Users/ivowings/Sync/Thesis/Datasources/Preprocessed/sentences/sentences_emscad.csv")
df_softskills = pd.read_csv("/Users/ivowings/Sync/Thesis/Datasources/Preprocessed/Skills/softskills.csv")
df_hardskills = pd.read_csv("/Users/ivowings/Sync/Thesis/Datasources/Preprocessed/Skills/emsi_skills.csv")

df_hardskills = df_hardskills[df_hardskills['type']== 'Hard Skill']
df_hardskills = pd.DataFrame(df_hardskills['name'])

#soft_skill,soft_skill_lemmatized,soft_skill_no_stopwords
columnname = 'soft_skill_lemmatized'
df_softskills = df_softskills[[columnname]]

#Renameing the skill column names and merging afterwards
df_softskills = df_softskills.rename(columns={columnname: 'skill'})
df_hardskills = df_hardskills.rename(columns={'name': 'skill'})
df_skills = pd.concat([df_softskills, df_hardskills])
df_skills = df_skills.drop_duplicates()
df_skills

#Creating all the possible grams for each sentence
allgrams = []

#This tokenizer immediately removes punctuation and special characters from the sentence
tokenizer = RegexpTokenizer(r'\w+')

#,sentence,sentence_lemmatized,sentence_no_stopwords

for sentence in tqdm(df_sentences['sentence_lemmatized']):
    tokenizedsentence = tokenizer.tokenize(str(sentence))
    
    #Getting all possible n grams of the sentence
    #for n in range(1,len(tokenizedsentence)+1):
    
    #getting up to tri grams for each sentence
    for n in range(1,4):
        grams = ngrams(tokenizedsentence,n)
        for gram in grams:
            allgrams.append(str(gram))

allgrams = pd.DataFrame(allgrams)
allgrams = allgrams.rename(columns={0:'allgrams'})
print(allgrams.shape[0])

#Initializing the keyword processor
keyword_processor = KeywordProcessor(case_sensitive=False)

#Adding all the skills to the processor
for skill in tqdm(df_skills['skill']):
    keyword_processor.add_keyword(skill)

def searcher(row):
    #check if the words are in the row and return a True or False instead of the actual word
    boolean = bool(keyword_processor.extract_keywords(row))
    return boolean

tqdm.pandas()
allgrams['contains_skill'] = allgrams['allgrams'].progress_apply(searcher)

#Only selecting the ngrams which contain a skill
allgrams = allgrams[allgrams.contains_skill == True]
#cleaning up
allgrams['allgrams'] = allgrams['allgrams'].astype(str)
allgrams['allgrams'] = allgrams['allgrams'].str.replace(r"[(),.']", '')
allgrams['allgrams'] = allgrams['allgrams'].str.strip()
allgrams = allgrams.drop(columns=['contains_skill'])

#Some softskills match multiple times, therefore removing the duplicates
allgrams = allgrams.drop_duplicates().reset_index(drop=True)

#removing any nans
allgrams.dropna(subset=['allgrams'],inplace=True)

allgrams.to_csv("/Users/ivowings/Sync/Thesis/Datasources/Preprocessed/n-grams/emscad_trigrams_lemmatized_taxonomy.csv", quoting=csv.QUOTE_NONNUMERIC,index=False)
allgrams

 24%|██▎       | 6786/28709 [00:00<00:00, 67854.56it/s]

12758752


100%|██████████| 12758752/12758752 [01:35<00:00, 132970.81it/s]
<ipython-input-3-df972771df2a>:62: FutureWarning: The default value of regex will change from True to False in a future version.
  allgrams['allgrams'] = allgrams['allgrams'].str.replace(r"[(),.']", '')


,allgrams
0,team
1,small team
2,team of
3,a small team
4,small team of
...,...
308888,creatively and step
308889,figure problem
308890,to figure problem
308891,figure problem out


In [4]:
df_sentences = pd.read_csv("/Users/ivowings/Sync/Thesis/Datasources/Preprocessed/sentences/sentences_emscad.csv")
df_softskills = pd.read_csv("/Users/ivowings/Sync/Thesis/Datasources/Preprocessed/Skills/softskills.csv")
df_hardskills = pd.read_csv("/Users/ivowings/Sync/Thesis/Datasources/Preprocessed/Skills/emsi_skills.csv")

df_hardskills = df_hardskills[df_hardskills['type']== 'Hard Skill']
df_hardskills = pd.DataFrame(df_hardskills['name'])

#soft_skill,soft_skill_lemmatized,soft_skill_no_stopwords
columnname = 'soft_skill_no_stopwords'
df_softskills = df_softskills[[columnname]]

#Renameing the skill column names and merging afterwards
df_softskills = df_softskills.rename(columns={columnname: 'skill'})
df_hardskills = df_hardskills.rename(columns={'name': 'skill'})
df_skills = pd.concat([df_softskills, df_hardskills])
df_skills = df_skills.drop_duplicates()
df_skills

#Creating all the possible grams for each sentence
allgrams = []

#This tokenizer immediately removes punctuation and special characters from the sentence
tokenizer = RegexpTokenizer(r'\w+')

#,sentence,sentence_lemmatized,sentence_no_stopwords

for sentence in tqdm(df_sentences['sentence_no_stopwords']):
    tokenizedsentence = tokenizer.tokenize(str(sentence))
    
    #Getting all possible n grams of the sentence
    #for n in range(1,len(tokenizedsentence)+1):
    
    #getting up to tri grams for each sentence
    for n in range(1,4):
        grams = ngrams(tokenizedsentence,n)
        for gram in grams:
            allgrams.append(str(gram))

allgrams = pd.DataFrame(allgrams)
allgrams = allgrams.rename(columns={0:'allgrams'})
print(allgrams.shape[0])

#Initializing the keyword processor
keyword_processor = KeywordProcessor(case_sensitive=False)

#Adding all the skills to the processor
for skill in tqdm(df_skills['skill']):
    keyword_processor.add_keyword(skill)

def searcher(row):
    #check if the words are in the row and return a True or False instead of the actual word
    boolean = bool(keyword_processor.extract_keywords(row))
    return boolean

tqdm.pandas()
allgrams['contains_skill'] = allgrams['allgrams'].progress_apply(searcher)

#Only selecting the ngrams which contain a skill
allgrams = allgrams[allgrams.contains_skill == True]
#cleaning up
allgrams['allgrams'] = allgrams['allgrams'].astype(str)
allgrams['allgrams'] = allgrams['allgrams'].str.replace(r"[(),.']", '')
allgrams['allgrams'] = allgrams['allgrams'].str.strip()
allgrams = allgrams.drop(columns=['contains_skill'])

#Some softskills match multiple times, therefore removing the duplicates
allgrams = allgrams.drop_duplicates().reset_index(drop=True)

#removing any nans
allgrams.dropna(subset=['allgrams'],inplace=True)

allgrams.to_csv("/Users/ivowings/Sync/Thesis/Datasources/Preprocessed/n-grams/emscad_trigrams_nostopwords_taxonomy.csv", quoting=csv.QUOTE_NONNUMERIC,index=False)
allgrams

 32%|███▏      | 9185/28860 [00:00<00:00, 91846.36it/s]

7607899


100%|██████████| 7607899/7607899 [01:03<00:00, 120493.54it/s]
<ipython-input-4-fa3654302a36>:62: FutureWarning: The default value of regex will change from True to False in a future version.
  allgrams['allgrams'] = allgrams['allgrams'].str.replace(r"[(),.']", '')


,allgrams
0,team
1,small team
2,team editors
3,work small team
4,small team editors
...,...
406592,think creatively step
406593,creatively step outside
406594,eager learn great
406595,figure problems
